# CRS

Coordinates Reference System

Les cartes de géographies tentent de représenter en 2D des points sur la terre : il faut pouvoir associer 2 coordonnées à tout point sur la surface de la terre.

Ce n'est pas simple du tout car la terre, ce n'est pas ça :

![terre sphere](https://upload.wikimedia.org/wikipedia/commons/d/d6/Sciences_de_la_terre.svg)

mais plutôt ça : 

<img src="https://upload.wikimedia.org/wikipedia/commons/4/4a/Geoid_undulation_10k_scale.jpg" alt="terre biscornue" width="300px"/>

La façon classique de considérer la terre étant ça (un elllipsoïde) :

<img src="https://upload.wikimedia.org/wikipedia/commons/b/b5/OblateSpheroid.PNG" alt="ellipsoïde" width="300px"/>

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

## Représentation classique du monde

Le module [geodatasets](https://geodatasets.readthedocs.io/en/latest/index.html) contient quelques jeux de données, dont la terre. 

Installez le dans votre environnement virtuel et importez le.

In [ ]:
import geodatasets

In [ ]:
geodatasets.data

In [ ]:
path = geodatasets.get_path('naturalearth land')
monde = gpd.read_file(path)

In [ ]:
monde.head()

In [ ]:
# encodage du monde actuel (on y reviendra en détail plus tard)

monde.crs

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15)) 

monde.plot(ax=ax)

ax.set_title("WGS 84");
plt.show()

### Mercator

Le premier à réaliser ce prodige a été [mercator](https://fr.wikipedia.org/wiki/Projection_de_Mercator) qui a projeté une sphère sur un cylindre avec l'équateur comme milieu :

Son système de coordonnées est [là](https://epsg.io/3395)

In [ ]:
mercator = monde.to_crs(epsg=3395)

In [ ]:
mercator.crs

In [ ]:
mercator

In [ ]:
fig, ax = plt.subplots(figsize=(15, 25)) 

ax.set_ylim(-.2e8, .2e8)
mercator.plot(ax=ax)

ax.set_title("Mercator")
plt.show()

La projection de mercator déforme cependant énormément les contours loin de l'équateur, mais les angles sont conservés ce qui est crucial en navigation.

### WGS 84 (GPS en lat/lon)



In [ ]:
gps = monde.to_crs(epsg=4326)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15)) 

gps.plot(ax=ax)

ax.set_title("WGS 84");
plt.show()

Vous voyez qu'il y a des différences

### peters

Une dernière projection pour la route. Celle de [Peters](https://fr.wikipedia.org/wiki/Projection_de_Peters), souvent représentée la tête en bas. Elle respecte les surfaces.

Elle est définie [là](https://spatialreference.org/ref/sr-org/22/)


In [ ]:
peters = monde.to_crs("+proj=cea +lon_0=0 +lat_ts=45 +x_0=0 +y_0=0 +ellps=WGS84 +units=m +no_defs +axis=wsu")

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15)) 

peters.plot(ax=ax)

ax.set_title("Peters");
plt.show()

### conclusions

Tout est faux. Chaque système de coordonnées a des soucis quelquepart. Il faut donc utiliser celui qui en a le moins à l'endroit où l'on regarde.

http://metrocosm.com/compare-map-projections.html

## tutos 

* une [introduction aux systèmes de coordonnées](https://medium.com/@_FrancoisM/introduction-%C3%A0-la-manipulation-de-donn%C3%A9es-cartographiques-23b4e38d8f0f) avec une vidéo éclairante.
* un [autre tuto](
https://medium.com/cr%C3%A9ation-dune-app-cartographique-avec-firebase-vue/comprendre-les-coordinates-reference-system-crs-b67a88bce63c) très bien fait en français (avec de vrais morceaux de lol dedans en plus).
* [super vidéo](https://www.youtube.com/watch?v=xJyJlKbZFlc&list=PLewNEVDy7gq3DjrPDxGFLbHE4G2QWe8Qh&index=8) vous expliquant très bien tout ça (mais passez le en x1.5 sinon vous allez vous endormir). Toute la playlist est bien d'ailleurs.
* [un grop pdf](https://pubs.usgs.gov/bul/1532/report.pdf)

Les sites de références qui recencent les différentes projections :

* https://epsg.io/
* https://spatialreference.org/

### conclusion

Lorsque vous chargez des donnes il est **CRITIQUE** que vous leurs associez leur CRS, sinon aucune conversion ne sera possible.

Lorsque vous allez faire des calculs ou des graphiques, il est indispensable que toutes vos données géographiques soient avec le même système de coordonnées (CRS).

Si vous voulez créer les votres (on fera un essai ci-après), vous pouvez lire la doc :
* de geopandas : https://geopandas.org/projections.html
* des différents paramètres que l'on peut utiliser : https://proj.org/usage/projections.html

## Rotation de carte

On va montrer comment faire une rotation de carte pour expliciter qu'une carte c'est :
1. toujours pour que quelqu'un la regarde
2. elle doit avoir un but

exemple : 

* mercator pour les marins (elle respecte les angles)
* Peters qui respecte les surfaces réelle des pays (l'afrique, c'est grand !)

Surtout, son centre n'est qu'une convention :
    * carte européenne (greenwich)
    * carte américaine (centrée sur le milieu des USA)
    * carte chinoise (centrée sur le pacifique)

On va essayer de le voir en déplaçant le centre de la carte. Commençons par toruver notre carte avec le méridien de greenwitch en 0 :

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15)) 

(monde
     .plot(ax=ax)
)
ax.axvline(x=0, color="green")

plt.show()

Ses paramètres de projections sont :

In [ ]:
monde.crs

En [lisant la doc](https://proj.org/usage/projections.html) des paramètres de projection, on doit toucher au paramètre `pm` pour changer le centre vertical. 

On veut centrer la carte pour les USA, donc une longitude de -100 environ.

Sauf que si on change de repère sans faire attention : 

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15)) 

(monde
     .to_crs("+proj=longlat +pm=100 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
     .plot(ax=ax)
)
ax.axvline(x=0, color="green")
plt.show()

C'est la catastrophe. 

Tous les pays qui chevauchent les nouveaux bords sont détruits : ils ont des bords à gauche et à droite de la figure... 

Pour palier ça, il faut découper la carte pour que chaque région soit toujours d'un côté de la carte. Pour que l'on puisse faire plusieurs essais, on va découper la carte en tronçons de 10 de longitude.

### découpage en tronçons

on va créer des petites bandes de .02 de largeur sur toute la carte et les soustraire à notre carte. Ceci découpera nos pays par tronçons de 10 de longitude.

In [ ]:
from shapely.geometry import Polygon

In [ ]:
bandes = []
delta = .01
for pm in range(-180, 180, 10):
    bandes.append(Polygon([(pm - delta, -100), (pm - delta, 100), (pm + delta,100), (pm + delta, -100)]))


meridiens = gpd.GeoDataFrame({'geometry': gpd.GeoSeries(bandes)}, 
                             crs=monde.crs)

découpage avec la fonction [overlay](https://geopandas.org/set_operations.html) de geopandas  qui est ultra puissante (on y reviendra) :

In [ ]:
monde_découpé = gpd.overlay(monde, meridiens, how='difference')

En dessinant, on voit bien les nouveaux bords :

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15)) 

monde_découpé.plot(ax=ax)

plt.show()

On voit bien les découpages. 

Pour bien faire on ne devrait faire que les découpages nécessaires pour la rotation, histoire de ne pas dénaturer les pays plus que ça.

**Note** : un bout de la sibérie est déjà découpée par défaut dans cette carte.

#### carte USA :

On centre en -100 (paramètre `+pm=-100`)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15)) 

(monde_découpé
     .to_crs("+proj=longlat +pm=-100 +x_0=0 +y_0=0 +ellps=WGS84 +units=m +no_defs")
     .plot(ax=ax)
)
ax.axvline(x=0, color="green")

plt.show()

#### carte chine

On centre en +110 (paramètre `+pm=-110`)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15)) 

(monde_découpé
     .to_crs("+proj=longlat +pm=110 +x_0=0 +y_0=0 +ellps=WGS84 +units=m +no_defs")
     .plot(ax=ax)
)

plt.show()

## Cartopy

<https://scitools.org.uk/>